In [279]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm
from warnings import filterwarnings

from delta_heding import delta_hedging

np.set_printoptions(precision=10, suppress=True)

In [69]:
'''Parameters'''
inputs = {
    's0' : 49,
    'k' : 50,
    'rf' : 0.05,
    'sigma' : 0.2,
    'imvol' : 0.2,
    'T' : 1,
    'mu' : 0.13,
    'q' : 0,
    'interval' : 1/52,
    'time_step' : 20,
    'simul_num' : 10000,
    'num_shares' : 100000
}

In [72]:
dh = delta_hedging(inputs)
dh.hedging_cost().mean()

2.4375090948462352

In [108]:
stock_path = dh.stock_path()
stock_path

array([[49.        , 49.        , 49.        , ..., 49.        ,
        49.        , 49.        ],
       [50.00072171, 49.1339912 , 49.73590216, ..., 46.83950973,
        48.81382304, 49.75173932],
       [49.44015818, 49.84700501, 48.38279151, ..., 47.36581304,
        48.41763696, 50.36473286],
       ...,
       [60.53628158, 56.19604059, 57.71828454, ..., 41.0772936 ,
        57.05917902, 54.82043891],
       [62.20802881, 55.8599315 , 58.16261092, ..., 39.69770014,
        57.09121606, 57.64300761],
       [62.69229552, 55.63181124, 59.06276658, ..., 40.36293711,
        56.98690506, 58.91838343]])

In [109]:
delta_path = np.round(dh.path_delta(stock_path),3)
delta_path

array([[0.522, 0.522, 0.522, ..., 0.522, 0.522, 0.522],
       [0.584, 0.527, 0.567, ..., 0.371, 0.505, 0.568],
       [0.544, 0.571, 0.471, ..., 0.4  , 0.473, 0.606],
       ...,
       [1.   , 0.999, 1.   , ..., 0.   , 1.   , 0.992],
       [1.   , 1.   , 1.   , ..., 0.   , 1.   , 1.   ],
       [1.   , 1.   , 1.   , ..., 0.   , 1.   , 1.   ]])

In [211]:
delta_path_shares = delta_path * inputs['num_shares']
delta_path_shares

array([[ 52200.,  52200.,  52200., ...,  52200.,  52200.,  52200.],
       [ 58400.,  52700.,  56700., ...,  37100.,  50500.,  56800.],
       [ 54400.,  57100.,  47100., ...,  40000.,  47300.,  60600.],
       ...,
       [100000.,  99900., 100000., ...,      0., 100000.,  99200.],
       [100000., 100000., 100000., ...,      0., 100000., 100000.],
       [100000., 100000., 100000., ...,      0., 100000., 100000.]])

In [214]:
delta_path_shares_ch = np.vstack([delta_path_shares[0], delta_path_shares[1:] - delta_path_shares[:-1]])
delta_path_shares_ch

array([[ 52200.,  52200.,  52200., ...,  52200.,  52200.,  52200.],
       [  6200.,    500.,   4500., ..., -15100.,  -1700.,   4600.],
       [ -4000.,   4400.,  -9600., ...,   2900.,  -3200.,   3800.],
       ...,
       [     0.,    500.,    100., ...,   -200.,    400.,    900.],
       [     0.,    100.,      0., ...,      0.,      0.,    800.],
       [     0.,      0.,      0., ...,      0.,      0.,      0.]])

In [218]:
delta_path_cost_shares_ch = np.round((delta_path_shares_ch * stock_path / 1000), 1)
delta_path_cost_shares_ch

array([[2557.8, 2557.8, 2557.8, ..., 2557.8, 2557.8, 2557.8],
       [ 310. ,   24.6,  223.8, ..., -707.3,  -83. ,  228.9],
       [-197.8,  219.3, -464.5, ...,  137.4, -154.9,  191.4],
       ...,
       [   0. ,   28.1,    5.8, ...,   -8.2,   22.8,   49.3],
       [   0. ,    5.6,    0. , ...,    0. ,    0. ,   46.1],
       [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ]])

In [285]:
delta_path_cumcost = np.repeat(delta_path_cost_shares_ch[0], stock_path.shape[0]).reshape(-1, stock_path.shape[1])
upper = (1 + inputs['rf']*1/52)

for x in range(0, delta_path_cost_shares_ch.shape[1]):
    for idx in range(delta_path_cost_shares_ch[:,x].shape[0]-1):
        delta_path_cumcost[idx+1, x] = np.round(delta_path_cumcost[idx,x] * upper + delta_path_cost_shares_ch[idx+1, x])



In [286]:
delta_path_cumcost

array([[2557.8, 2557.8, 2557.8, ..., 2557.8, 2557.8, 2557.8],
       [2870. , 2585. , 2784. , ..., 1853. , 2477. , 2789. ],
       [2675. , 2807. , 2322. , ..., 1992. , 2324. , 2983. ],
       ...,
       [5237. , 5226. , 5222. , ...,  208. , 5226. , 5159. ],
       [5242. , 5237. , 5227. , ...,  208. , 5231. , 5210. ],
       [5247. , 5242. , 5232. , ...,  208. , 5236. , 5215. ]])

In [294]:
int_cost = np.round(delta_path_cumcost * (1/52 * 0.05), 1)
int_cost

array([[2.5, 2.5, 2.5, ..., 2.5, 2.5, 2.5],
       [2.8, 2.5, 2.7, ..., 1.8, 2.4, 2.7],
       [2.6, 2.7, 2.2, ..., 1.9, 2.2, 2.9],
       ...,
       [5. , 5. , 5. , ..., 0.2, 5. , 5. ],
       [5. , 5. , 5. , ..., 0.2, 5. , 5. ],
       [5. , 5. , 5. , ..., 0.2, 5. , 5. ]])

In [304]:
index0_df = pd.DataFrame(data=[stock_path[:,0], delta_path[:,0], delta_path_shares_ch[:,0], delta_path_cost_shares_ch[:,0], delta_path_cumcost[:,0], int_cost[:,0]],\
                         index=['Stock_Price', 'Delta', 'Shares Purchased', 'Cost of Shares Purchased', 'Cummulative Cost', 'Interest Cost']).T

#index0_df
print('mean : ', np.round(index0_df['Cost of Shares Purchased'].mean(), 1), sep=' ')

mean :  246.4


In [302]:
index0_df['Cost of Shares Purchased'].mean()

246.44285714285715

In [240]:
upper = (1 + inputs['rf']*1/52)
upper * 2557.8 - 308

2252.259423076923

In [ ]:
check

In [222]:
2557.8 * (1/52 * 0.05 + 1) - 308

2252.259423076923

In [223]:
2252.3 * (1/52 * 0.05 + 1) - 274.7

1979.7656730769233

In [130]:
delta_path_ch = np.vstack([delta_path[0], delta_path[1:] - delta_path[:-1]])
delta_path_ch

array([[ 0.522,  0.522,  0.522, ...,  0.522,  0.522,  0.522],
       [ 0.062,  0.005,  0.045, ..., -0.151, -0.017,  0.046],
       [-0.04 ,  0.044, -0.096, ...,  0.029, -0.032,  0.038],
       ...,
       [ 0.   ,  0.005,  0.001, ..., -0.002,  0.004,  0.009],
       [ 0.   ,  0.001,  0.   , ...,  0.   ,  0.   ,  0.008],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ]])

In [131]:
delta_path_ch * inputs['num_shares']

array([[ 52200.,  52200.,  52200., ...,  52200.,  52200.,  52200.],
       [  6200.,    500.,   4500., ..., -15100.,  -1700.,   4600.],
       [ -4000.,   4400.,  -9600., ...,   2900.,  -3200.,   3800.],
       ...,
       [     0.,    500.,    100., ...,   -200.,    400.,    900.],
       [     0.,    100.,      0., ...,      0.,      0.,    800.],
       [     0.,      0.,      0., ...,      0.,      0.,      0.]])

In [206]:
6400 * 48.125

308000.0

In [ ]:
for row in delta_path:
    

In [137]:
delta_path_ch_cost = np.round((delta_path_ch * inputs['num_shares'] * stock_path) / 1000, 1)
delta_path_ch_cost

array([[2557.8, 2557.8, 2557.8, ..., 2557.8, 2557.8, 2557.8],
       [ 310. ,   24.6,  223.8, ..., -707.3,  -83. ,  228.9],
       [-197.8,  219.3, -464.5, ...,  137.4, -154.9,  191.4],
       ...,
       [   0. ,   28.1,    5.8, ...,   -8.2,   22.8,   49.3],
       [   0. ,    5.6,    0. , ...,    0. ,    0. ,   46.1],
       [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ]])

In [181]:
np.round(delta_path_ch_cost.cumsum(axis=0), 1)

array([[2557.8, 2557.8, 2557.8, ..., 2557.8, 2557.8, 2557.8],
       [2867.8, 2582.4, 2781.6, ..., 1850.5, 2474.8, 2786.7],
       [2670. , 2801.7, 2317.1, ..., 1987.9, 2319.9, 2978.1],
       ...,
       [5175.3, 5159.5, 5154. , ...,  178.2, 5160.7, 5091.9],
       [5175.3, 5165.1, 5154. , ...,  178.2, 5160.7, 5138. ],
       [5175.3, 5165.1, 5154. , ...,  178.2, 5160.7, 5138. ]])

In [185]:
2557.8*(1+0.05/52) - 2252.3

307.95942307692303

In [189]:
delta_path * inputs['num_shares'] 

array([[ 52200.,  52200.,  52200., ...,  52200.,  52200.,  52200.],
       [ 58400.,  52700.,  56700., ...,  37100.,  50500.,  56800.],
       [ 54400.,  57100.,  47100., ...,  40000.,  47300.,  60600.],
       ...,
       [100000.,  99900., 100000., ...,      0., 100000.,  99200.],
       [100000., 100000., 100000., ...,      0., 100000., 100000.],
       [100000., 100000., 100000., ...,      0., 100000., 100000.]])

In [187]:
308 * (1/52 * 0.05)

0.29615384615384616

In [186]:
2557.8 - 2252.3

305.5

In [167]:
(0.05 * (1/52) + 1) * 2252.3 - 274.8

1979.6656730769234

In [143]:
2557.8 * np.exp(2.5) - 308

30852.383052687346

In [107]:
dh.stock_path()

array([[49.        , 49.        , 49.        , ..., 49.        ,
        49.        , 49.        ],
       [48.80996289, 47.50632208, 47.44643131, ..., 47.21358382,
        50.18212083, 46.82443005],
       [48.07108216, 47.27932705, 48.71229157, ..., 45.70690829,
        50.77021185, 46.88929914],
       ...,
       [62.21978728, 43.67475246, 45.8479696 , ..., 51.73495004,
        48.83997365, 51.41686824],
       [65.08140814, 45.42376634, 45.9673953 , ..., 52.3353258 ,
        49.50229101, 50.35652774],
       [64.50525662, 45.35932124, 46.68778913, ..., 50.15900258,
        50.75476336, 50.6317454 ]])

In [103]:
delta_path * stock_path

array([2557.8, 2827.3, 2665.6])

In [104]:
2827.3-2557.8

269.5

In [91]:
25578000

25578000

In [88]:
52200 / 2557.8

20.408163265306122

In [87]:
1 - delta_path

array([[0.478, 0.478, 0.478, ..., 0.478, 0.478, 0.478],
       [0.423, 0.571, 0.476, ..., 0.329, 0.659, 0.385],
       [0.456, 0.62 , 0.424, ..., 0.288, 0.617, 0.421],
       ...,
       [0.62 , 0.8  , 1.   , ..., 0.   , 0.227, 0.981],
       [0.943, 0.878, 1.   , ..., 0.   , 0.392, 1.   ],
       [1.   , 1.   , 1.   , ..., 0.   , 1.   , 1.   ]])

In [10]:

class delta_hedging_test:

    def __init__(self, **input):
 
        self.s0         = input['s0']               # 현재가격
        self.k          = input['k']                # 행사가격
        self.rf         = input['rf']               # 무위험이자율
        self.sigma      = input['sigma']            # 변동성
        self.mu         = input['mu']               # 평균수익률
        self.T          = input['T']                # 투자기간
        self.q          = input['q']                # 배당금 
        self.iv         = input['imvol']            # 내재변동성

        self.interval   = input['interval']         # T의 기간 텀
        self.time_step  = input['time_step']        # Investment Horizon
        self.simul      = input['simul_num']        # 시뮬레이션 횟수

        if self.interval == 'weekly':               
            self.dt = self.T / 52                   
        elif self.interval == 'daily':
            self.dt = self.T / 250
        else:
            self.dt = input['interval']
        
        self.tau        = self.time_step * self.dt
        self.tau_arr    = (np.flip(np.arange(0, self.time_step+1)) * self.dt).reshape(-1,1)

In [34]:
class test:
    def __init__ (self, **kwargs):
        for key, values in kwargs.items():
            globals()['{}'.format(self.key)] = values
            print(f'{key} : {values}')

In [67]:
class test:
    def __init__ (self, s0, k, r, sigma, T, mu, interval **input):
        self.s0 = s0
        self.k  = k
    
        self.__dict__.update(input)

In [ ]:
)

In [64]:
a = test(s0 = 49,
    k = 50,
    rf = 0.05,
    sigma = 0.2,
    imvol = 0.2,
    T = 1,
    mu = 0.13,
    q = 0,
    interval = 1/52,
    time_step = 20,
    simul_num = 10000,
    num_shares = 100000)

{'s0': 49, 'k': 50, 'rf': 0.05, 'sigma': 0.2, 'imvol': 0.2, 'T': 1, 'mu': 0.13, 'q': 0, 'interval': 0.019230769230769232, 'time_step': 20, 'simul_num': 10000, 'num_shares': 100000}
12


In [65]:
a.s0

49

In [30]:
for key, values in inputs.items():
            print(f'{key} : {values}')

s0 : 49
k : 50
rf : 0.05
sigma : 0.2
imvol : 0.2
T : 1
mu : 0.13
q : 0
interval : 0.019230769230769232
time_step : 20
simul_num : 10000
num_shares : 100000


In [36]:
test(inputs)

TypeError: __init__() takes 1 positional argument but 2 were given

In [14]:
delta_hedging_test(inputs)

TypeError: __init__() takes 1 positional argument but 2 were given